1/ Importation des données TC et TITMC 

2/ Import et traitement du fichier des prénoms 

3/ Ajouter le genre des personnes 

4/ Codification des adresses 

5/ Ajout et traitement du code commune

6/ Retraiter et simplifier les qualités 

In [ ]:
# Lecture de la base inpi
import os
import s3fs
import pandas as pd
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/inpi"
FILE_KEY_S3 = "data_rep_pm_2017.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8")

In [ ]:
df_rep_pm.shape # 9 751 833

In [ ]:
df_rep_pm = df_rep_pm.rename({"code greffe": "code_greffe"}, axis=1)

In [ ]:
df_rep_pm = df_rep_pm.loc[:, ["code_greffe", "siren", "denomination", "forme_juridique_x", "qualité",
"nom_patronymique", "nom_usage", "prénoms", "type", "date_naissance", "ville_naissance", "adresse_ligne1", "adresse_ligne2",
"adresse_ligne3", "code_postal", "ville", "code_commune", "pays", "id_représentant"]]

In [ ]:
df_rep_pm = df_rep_pm.rename({"forme_juridique_x": "forme_juridique", "prénoms": "prenoms", "qualité": "qualite", "id_représentant": "id_representant"}, axis=1)

In [ ]:
df_rep_pm.head(10)

In [ ]:
df_rep_pm["qualite"].isna().sum() # 246 926

In [ ]:
# Traitement des cas où le nom et les prénoms sont manquants

re2 = "(\sNEE\s[A-Za-z\-]+)\s([A-Za-z\-\.\s]+)"


In [ ]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "-", "prenoms"] = ""

In [ ]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "", :].shape # 4 448

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") & (df_rep_pm["type"] == "P.Physique"), :].shape # 4 448

In [ ]:
re1 = "(INDIVISION SUCCESSORALE DE M\.\s|\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s|REPRESENTEE\sPAR\sMONSIEUR\.?\s|REPRESENTEE\sPAR\s)"
df_rep_pm["top_civilite"] = df_rep_pm["nom_patronymique"].str.contains(re1, regex=True)

In [ ]:
df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique"].head()

In [ ]:
df_rep_pm["contenu_civilite"] = df_rep_pm["nom_patronymique"].str.extract(re1)

In [ ]:
df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "contenu_civilite"].head()

In [ ]:
df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique"]

In [ ]:
df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique2"].str.replace(re1,"")

In [ ]:
from pandas import option_context
with option_context('display.max_colwidth', 400):
    print(df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique2"].head(10))

In [ ]:
# on repartit les noms et les prénoms dans les bonnes variables
# on a modifié un peu le code de l'insee pour enlever les representee par, ici aussi car on va essayer de prendre en compte les particules
particule = "(DU\s[A-Za-z]+|DE\sLA\s[A-Za-z]+|[A-Za-z]+\sDU\s[A-Za-z]+|[A-Za-z]+\sDE\sLA\s[A-Za-z]+|DE\s[A-Za-z]+|[A-Za-z]+\sDE\s[A-Za-z]+)"
#particule = "(DU\s|DE\sLA\s|\sDU\s|\sDE\sLA\s|DE\s|\sDE\s)"
df_rep_pm["top_particule"] = df_rep_pm["nom_patronymique"].str.contains(particule, regex=True)

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") | (df_rep_pm["prenoms"].isna()),:].shape # 1093022

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == ""),:].shape # 4448, mais ce sont tous des cas différents, donc on n'a pas envie de séparer nom et prénoms comme le fait l'insee

In [ ]:
df_rep_pm.loc[(df_rep_pm["top_particule"] == True) & (df_rep_pm["prenoms"] == ""), "nom_patronymique2"].head(10)

In [ ]:
df_rep_pm.loc[df_rep_pm["nom_usage"].isna(), :].head() # pas d'info parfois quand il y a l'info nom patronymique et prénoms 

In [99]:
# nettoyage de la variable qualite 
df_rep_pm["qualite"].value_counts()

Gérant                                  2413218
Associé                                 2174834
Gérant                                  1112522
Associé indéfiniment responsable        1089112
gérant                                   633900
                                         ...   
Adhérent                                      1
Directeur technique et administratif          1
Comptable                                     1
Contrôleur des comptes adjoint                1
Mandataire général                            1
Name: qualite, Length: 371, dtype: int64

In [100]:
df_rep_pm["qualite2"] = df_rep_pm["qualite"]

In [104]:
df_rep_pm["qualite2"] = df_rep_pm["qualite2"].str.lower()
df_rep_pm["qualite2"] = df_rep_pm["qualite2"].str.strip()

In [105]:
df_rep_pm["qualite2"].value_counts()

gérant                                                                                                              4159640
associé                                                                                                             2174857
associé indéfiniment responsable                                                                                    1089112
président                                                                                                            578505
commissaire aux comptes titulaire                                                                                    249159
                                                                                                                     ...   
adhérent                                                                                                                  1
directeur technique et administratif                                                                                      1
membre t

In [106]:
# nettoyage de la dénomination
df_rep_pm["denomination2"] = df_rep_pm["denomination"]

In [108]:
df_rep_pm["denomination2"] = df_rep_pm["denomination2"].str.lower()

In [109]:
df_rep_pm["denomination2"].head(10)

0                                    agir technologies
1                                                doras
2    societe immobiliere financiere d etudes et de ...
3    societe immobiliere financiere d etudes et de ...
4    societe immobiliere financiere d etudes et de ...
5    societe immobiliere financiere d etudes et de ...
6       societe nationale des etablissements piot pneu
7                                           richardson
8                                                samse
9                                                 bmra
Name: denomination2, dtype: object

In [110]:
# codification des adresses
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite', 'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance', 'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3', 'code_postal', 'ville', 'code_commune', 'pays', 'id_representant', 'top_civilite', 'contenu_civilite', 'nom_patronymique2', 'top_particule', 'qualite2', 'denomination2'], dtype='object')

In [112]:
df_rep_pm["ville"].head(10)

0           NaN
1           NaN
2    Carpentras
3         Paris
4    Carpentras
5    Strasbourg
6           NaN
7           NaN
8           NaN
9           NaN
Name: ville, dtype: object

In [113]:
df_rep_pm.loc[df_rep_pm["ville"] == "", :].shape

(0, 25)